<a href="https://colab.research.google.com/github/jufeitosa10/XAI_Segmentacao_Pacu/blob/main/Combinacao_Metodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#para pegar o que tem  no drive e poder usar

import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/XAI')

In [ ]:
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import numpy as np
import cv2
import torchvision.models.detection.anchor_utils
from torchvision.transforms import ToTensor, ToPILImage
import requests
from io import BytesIO
from PIL import Image, ExifTags, ImageFile
from torchvision.transforms import functional as F
from matplotlib import pyplot as plt
from torchvision.transforms import ToTensor, ToPILImage
import os
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor, MaskRCNN

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

def get_transform():
    transforms = []
    transforms.append(ToTensor())

    return Compose(transforms)

class LabelboxDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        width,height = img.size
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            if xmin >= 0 and xmax <= width and xmin <= xmax and ymin >= 0  and ymax <= height and ymin <= ymax:
                xmin_new = min(xmin, xmax)
                xmax_new = max(xmin, xmax)
                boxes.append([xmin_new, ymin, xmax_new, ymax])
            else:
                print(self.imgs[idx])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        keep = (boxes[:, 3]>boxes[:, 1]) & (boxes[:, 2]>boxes[:, 0])
        boxes = boxes[keep]
        labels = labels[keep]
        masks = masks[keep]
        area = area[keep]
        iscrowd = iscrowd[keep]

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)
ORIENTATION_ROTATION_MAP = {
    0: 0,
    1: 360,
    2: 360,
    3: 180,
    4: 180,
    5: 270,
    6: 270,
    7: 90,
    8: 90,
}

class URLDataset(object):
    def __init__(self, urls, transforms, scale=1.0):
        self.urls = urls
        self.transforms = transforms
        self.scale = scale
    def __getitem__(self, idx):

        image = Image.open(BytesIO(requests.get(self.urls[idx]).content)).convert("RGB")
        if image.getexif():
            exif = dict((ExifTags.TAGS[k], v) for k, v in image._getexif().items() if k in ExifTags.TAGS)
            orient = exif.get('Orientation', 0)
            if orient > 0:
                image = image.rotate(ORIENTATION_ROTATION_MAP[orient], expand=True)
        w,h = image.size
        scaled_w = int(np.ceil(w * self.scale))
        scaled_h = int(np.ceil(h * self.scale))

        image = image.resize((scaled_w, scaled_h), Image.ANTIALIAS)

        # convert the PIL Image into a numpy array
        mask = np.zeros((scaled_h, scaled_w), np.float32)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        width,height = image.size
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            if xmin >= 0 and xmax <= width and xmin <= xmax and ymin >= 0  and ymax <= height and ymin <= ymax:
                xmin_new = min(xmin, xmax)
                xmax_new = max(xmin, xmax)
                boxes.append([xmin_new, ymin, xmax_new, ymax])
            else:
                print(self.urls[idx])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        if len(boxes):
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        else:
            area = 0
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

#         keep = (boxes[:, 3]>boxes[:, 1]) & (boxes[:, 2]>boxes[:, 0])
#         boxes = boxes[keep]
#         labels = labels[keep]
#         masks = masks[keep]
#         area = area[keep]
#         iscrowd = iscrowd[keep]

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target, self.urls[idx]

    def __len__(self):
        return len(self.urls)

class FolderDataset(object):
    def __init__(self, urls, transforms, scale=1.0):
        self.urls = urls
        self.transforms = transforms
        self.scale = scale
    def __getitem__(self, idx):
        image = Image.open(self.urls[idx]).convert("RGB")
        if image.getexif():
            exif = dict((ExifTags.TAGS[k], v) for k, v in image.getexif().items() if k in ExifTags.TAGS)
            orient = exif.get('Orientation', 0)
            if orient > 0:
                image = image.rotate(ORIENTATION_ROTATION_MAP[orient], expand=True)
        w,h = image.size
        scaled_w = int(np.ceil(w * self.scale))
        scaled_h = int(np.ceil(h * self.scale))

        image = image.resize((scaled_w, scaled_h), Image.ANTIALIAS)

        # convert the PIL Image into a numpy array
        mask = np.zeros((scaled_h, scaled_w), np.float32)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        width,height = image.size
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            if xmin >= 0 and xmax <= width and xmin <= xmax and ymin >= 0  and ymax <= height and ymin <= ymax:
                xmin_new = min(xmin, xmax)
                xmax_new = max(xmin, xmax)
                boxes.append([xmin_new, ymin, xmax_new, ymax])
            else:
                print(self.urls[idx])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        if len(boxes):
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        else:
            area = 0
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

#         keep = (boxes[:, 3]>boxes[:, 1]) & (boxes[:, 2]>boxes[:, 0])
#         boxes = boxes[keep]
#         labels = labels[keep]
#         masks = masks[keep]
#         area = area[keep]
#         iscrowd = iscrowd[keep]

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target, self.urls[idx]

    def __len__(self):
        return len(self.urls)
def get_maskrcnn_model(num_classes, FROZEN_BLOCKS):
    resnet = torchvision.models.resnet18(pretrained=True)
    backbone = torch.nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1,
                                  resnet.layer2, resnet.layer3, resnet.layer4)
    # Fix blocks
    for p in backbone[0].parameters():
        p.requires_grad = False
    for p in backbone[1].parameters():
        p.requires_grad = False
    if FROZEN_BLOCKS >= 3:
        for p in backbone[6].parameters():
            p.requires_grad = False
    if FROZEN_BLOCKS >= 2:
        for p in backbone[5].parameters():
            p.requires_grad = False
    if FROZEN_BLOCKS >= 1:
        for p in backbone[4].parameters():
            p.requires_grad = False

    def set_bn_fix(m):
        classname = m.__class__.__name__
        if classname.find('BatchNorm') != -1:
            for p in m.parameters(): p.requires_grad = False
    backbone.apply(set_bn_fix)

    backbone.out_channels = 512
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                        aspect_ratios=((0.5, 1.0, 2.0),))
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                              output_size=7,
                                              sampling_ratio=2)

    mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                                                output_size=14,
                                                                sampling_ratio=2)
    return MaskRCNN(backbone,
                        num_classes=num_classes,
                        rpn_anchor_generator=anchor_generator,
                        box_roi_pool=roi_pooler,
                        box_detections_per_img=10,
                        mask_roi_pool=mask_roi_pooler)
def get_maskrcnn_mobilenetv2_model(num_classes):
    backbone = torchvision.models.mobilenet_v2(pretrained=True).features
    backbone.out_channels = 1280

    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                        aspect_ratios=((0.5, 1.0, 2.0),))
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                              output_size=7,
                                              sampling_ratio=2)

    mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                                                output_size=14,
                                                                sampling_ratio=2)
    return MaskRCNN(backbone,
                        num_classes=num_classes,
                        rpn_anchor_generator=anchor_generator,
                        box_roi_pool=roi_pooler,
                        box_detections_per_img=10,
                        mask_roi_pool=mask_roi_pooler)
def get_coloured_mask(mask, label=None):
    """
    random_colour_masks
      parameters:
        - image - predicted masks
      method:
        - the masks of each predicted object is given random colour for visualization
    """
    colours = [
        [80, 220, 100], # index 0 is yellow
        [255, 0, 0], # index 1 is redtransforms
        [0, 64, 255], # index 2 is blue
        [0, 64, 255], # index 2 is blue
        [255, 0, 255],
        [255,255,255], # index
        # [157,32,161],
        # [4,44,213],
    ]
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    if label is not None:
        color = colours[label]
    else:
        color = colours[random.randrange(0,len(colours))]
    r[mask == 1], g[mask == 1], b[mask == 1] = color
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask


def display_pred(image, prediction, thres=0.50, alpha = 0.50):
    image = np.array(image)


    labels = prediction['labels'].cpu().clone().detach().numpy()
    masks = prediction['masks'].cpu().clone().detach().numpy()
    boxes = prediction["boxes"].cpu().clone().detach().numpy()
    scores = prediction["scores"].cpu().clone().detach().numpy()
    #max_score_idx = scores.argsort()[-top:][::-1]

    rect_th=1


    for idx,score  in enumerate(scores):
        box, mask, label= boxes[idx], np.copy(masks[idx]),labels[idx]
        if score >= thres:
            mask[np.where(mask>thres)] = 1
            mask[np.where(mask<thres)] = 0

            rgb_mask = get_coloured_mask(mask, label).squeeze().transpose(0,2,1)

            image = cv2.addWeighted(image, 1, rgb_mask, alpha, 0)

            start = (box[0], box[3])
            end = (box[2],box[1])
            cv2.rectangle(image, start, end,color=(255, 0, 0), thickness=rect_th)

    return image

def display_black_mask(image, prediction, thres=0.50, alpha = 0.50):
    image = np.array(image)
    zeros = np.zeros(image.shape)
    labels = prediction['labels'].cpu().clone().detach().numpy()
    masks = prediction['masks'].cpu().clone().detach().numpy()
    boxes = prediction["boxes"].cpu().clone().detach().numpy()
    scores = prediction["scores"].cpu().clone().detach().numpy()
    #max_score_idx = scores.argsort()[-top:][::-1]
    rect_th=1
    for idx,score  in enumerate(scores):
        box, mask, label= boxes[idx], np.copy(masks[idx]),labels[idx]
        if score >= thres:
            mask[np.where(mask>thres)] = 1
            mask[np.where(mask<thres)] = 0
            m = np.uint8(mask.transpose(1,2,0)*image)
            zeros = cv2.addWeighted(np.uint8(zeros), 1, m, 1, 0)

            start = (box[0], box[3])
            end = (box[2],box[1])
            cv2.rectangle(zeros, start, end,color=(255, 0, 0), thickness=rect_th)

    return zeros

def display_white_mask(image, prediction, thres=0.50, alpha = 0.50):
    image = np.array(image)
    zeros = np.zeros(image.shape)
    labels = prediction['labels'].cpu().clone().detach().numpy()
    masks = prediction['masks'].cpu().clone().detach().numpy()
    boxes = prediction["boxes"].cpu().clone().detach().numpy()
    scores = prediction["scores"].cpu().clone().detach().numpy()
    #max_score_idx = scores.argsort()[-top:][::-1]
    rect_th=1
    for idx,score  in enumerate(scores):
        box, mask, label= boxes[idx], np.copy(masks[idx]),labels[idx]
        if score >= thres:
            mask[np.where(mask>thres)] = 1
            mask[np.where(mask<thres)] = 0
            m = np.uint8(image*mask.transpose(1,2,0))
            zeros = cv2.addWeighted(np.uint8(zeros), 1.0, m, 1, 0)

            start = (box[0], box[3])
            end = (box[2],box[1])
            #cv2.rectangle(zeros, start, end,color=(255, 0, 0), thickness=rect_th)
    zeros[np.where(zeros==0)] = 255
    return zeros

from PIL import ImageFile
import os

def check_resolutions(IMAGE_DIR):
    ImPar=ImageFile.Parser()

    dicta = {}
    i = 0
    for fi in os.listdir(IMAGE_DIR):
        with open(f"{IMAGE_DIR}/{fi}", "rb") as f:
            #print(fi)
            if ".csv" not in fi:
                ImPar=ImageFile.Parser()
                chunk = f.read(2048)
                count=2048
                while chunk != "" and chunk != None:

                    ImPar.feed(chunk)
                    if ImPar.image:
                        break
                    chunk = f.read(2048)
                    count+=2048
                dicta[ImPar.image.size] = fi
                i+=1
                #print(i)
    return dicta


#Combinações







In [ ]:
# !pip install pip==21.3.1
!pip install grad-cam

# Taken from the torchvision tutorial
# https://pytorch.org/vision/stable/auto_examples/plot_visualization_utils.html
# from prediction_utils import URLDataset, get_maskrcnn_model, get_transform, display_pred, display_black_mask, display_white_mask, get_coloured_mask, check_resolutions
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from time import time
import numpy as np
import cv2
import torch
from torchvision.models import resnet50


model = get_maskrcnn_model(num_classes=4, FROZEN_BLOCKS=1)
#original
#model.load_state_dict(torch.load('/resnet18_checkpoint_355.pth', map_location=torch.device("cpu")))

#modificado pela Ju - pega arquivo direto do Drive
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/XAI/resnet18_checkpoint_355.pth', map_location=torch.device("cpu")))

model = model.eval()
model = model.to(torch.device("cpu"))






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from torchvision.models.segmentation import deeplabv3_resnet50
import torch
import torch.functional as F
import numpy as np
import requests
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
import requests
from io import BytesIO
from PIL import Image, ExifTags, ImageFile
from torchvision.transforms import functional as F

arq = open("/content/drive/MyDrive/Colab Notebooks/XAI/nomes_imagens_peixes2.txt")
linhas = arq.readlines()

for linha in linhas:
    print(linha)

ckz4dow5r8eay0z9i9oef91zg

ckz4e0jvq8h73109bbpqs0fps

ckz4e2n4q8f530z8n9s3ubp7y

ckz4e3onq8gmi0z9i5c70ay45

ckz4e4vvo8i6i109bgzla6hms

ckz4e8ge48hhl0z9ifd9773xk

ckz4e9ebq8gg20z8ngztba1ez

ckz4e650k8ig1109b97k528nn

ckz4e72918ion109b5wbr5raw

ckz4ea3cz8jbf109bhn9y68q1

ckz4ebk6g8i8u0z9i60n60f6l

ckz4edumg8ing0z9ifni9g5t8

ckz4efrf38kem109b2057fb9a

ckz4egfr48kh7109bah7xf5az

ckz4fduq78ntw0z8n630rag87

ckz4ff6t28o3e0z8neshr668d

ckz4ffyse8o9s0z8n3zgmd73e

ckz4fhxu88ool0z8n6rwr1cpm

ckz4fiupe8rhx109bh3peeyhc

ckz4fzcn28sui0z8nakiacny4

ckz4g2msm8tqx0z8n6d9pb3iv

ckz4g4h098uag0z8nht9q26i8

ckz4g5tsc8um20z8n9i7a5xw6

ckz4g6h9v8x8e109b16pefhfo

ckz4g7nl08v2x0z8nbuek2vjk

ckz4g09s18vbm109bhm9pcd53

ckz4g9uib8x7y0z9i2x6dgzf8

ckz4g17ra8upt0z9ibj0bbdno

ckz4g51wk8wtj109bhzrydupk

ckz4g70yq8we70z9idki606ak

ckz4g39038vcp0z9i67o96efj

ckz4g88978v9h0z8n0bmhf6m2

ckz4gb3qw8yne109bd7k39lcl

ckz4gbrb08xqo0z9i6tencxws

ckz4gceez8xwm0z9ihjmr0f68

ckz4gd9eh8y3f0z9i649mbt2l

ckz4gds8l8zdg109b2j93eman

c

### Mapa de Saliência


In [ ]:
import torch
import torchvision
import  torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from time import time
import numpy as np
import cv2
import torch



#define transforms to preprocess input image into format expected by model
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
#inverse transform to get normalize image back to original form for visualization
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

#transforms to resize image to the size expected by pretrained model,
#convert PIL image to tensor, and
#normalize the image
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #normalize,
])





In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def saliency(img, model):
    #we don't need gradients w.r.t. weights for a trained model
    for param in model.parameters():
        param.requires_grad = False

    #set model in eval model
    model.eval()

    #transoform input PIL image to torch.Tensor and normalize
    input = transform(img)
    input.unsqueeze_(0)

    #we want to calculate gradient of higest score w.r.t. input
    #so set requires_grad to True for input
    input.requires_grad = True


    #forward pass to calculate predictions
    preds = model(input)

    score, indices = torch.max((preds[0]['masks']), 1)



    #-------colocado pela Ju
    score.mean().backward()
    #print(score.sum())


    #get max along channel axis
    slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)

    slc = (slc - slc.min())/(slc.max()-slc.min())
    #apply inverse transform on image

    #usa a normalização - tirei porque talvez estivesse atrapalhando o modelo do Fabrício
    '''with torch.no_grad():
        input_img = inv_normalize(input[0])'''

    masks = preds[0]['masks'][0].squeeze().detach().cpu().numpy()
    masks_uint8 = 255 * np.uint8(masks > 0.5)
    masks_float = np.float32(masks_uint8 > 0.5)

    copia_img =  np.float32(image.convert("RGB"))

    mascara_merge = np.zeros((copia_img.shape[0], copia_img.shape[1]))

    for p in preds[0]['masks']:
      t = p.squeeze().detach().cpu().numpy()
      for i in range(len(t)):
        for j in range(len(t[i])):
            if t[i][j]>0.5:
              mascara_merge[i][j] = 1

    return slc

### Grad-CAM

In [ ]:
# from prediction_utils import URLDataset, get_maskrcnn_model, get_transform, display_pred, display_black_mask, display_white_mask, get_coloured_mask, check_resolutions
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from time import time
import numpy as np
import cv2
import torch
from torchvision.models import resnet50



In [ ]:
class SegmentationModelOutputWrapper(torch.nn.Module):
    def __init__(self, model):
        super(SegmentationModelOutputWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model([F.to_tensor(x)])[0]["masks"]

In [ ]:
from pytorch_grad_cam import GradCAM
import random

class SemanticSegmentationTarget:
    def __init__(self, category, mask):
        self.category = category
        self.mask = torch.from_numpy(mask)

    def __call__(self, model_output):
        return (model_output['masks'][self.category, :, : ] * self.mask).sum()

###CNN Filters

https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
from PIL import Image
import json
%matplotlib inline

In [ ]:
def CnnFilters(imagem, model):


  model_weights =[]

  conv_layers = []

  model_children = list(model.backbone)

  counter = 0
  #append all the conv layers and their respective wights to the list
  for i in range(len(model_children)):
      if type(model_children[i]) == nn.Conv2d:
          counter+=1
          model_weights.append(model_children[i].weight)
          conv_layers.append(model_children[i])
      elif type(model_children[i]) == nn.Sequential:
          for j in range(len(model_children[i])):
              for child in model_children[i][j].children():
                  if type(child) == nn.Conv2d:
                      counter+=1
                      model_weights.append(child.weight)
                      conv_layers.append(child)


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)

  outputs = []
  names = []
  for layer in conv_layers[0:]:
      imagem = layer(imagem)
      outputs.append(imagem)
      names.append(str(layer))


  processed = []
  for feature_map in outputs:
      feature_map = feature_map.squeeze(0)
      gray_scale = torch.sum(feature_map,0)
      gray_scale = gray_scale / feature_map.shape[0]
      processed.append(gray_scale.data.cpu().numpy())

  result = outputs[0][0]
  return result

###Layer GradCAM

In [ ]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# from prediction_utils import URLDataset, get_maskrcnn_model, get_transform, display_pred, display_black_mask, display_white_mask, get_coloured_mask, check_resolutions
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from time import time
import numpy as np
import cv2
import torch
from torchvision.models import resnet50
from captum.attr import visualization as viz
from captum.attr import LayerGradCam, FeatureAblation, LayerActivation, LayerAttribution

In [ ]:
model = get_maskrcnn_model(num_classes=4, FROZEN_BLOCKS=1)

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/XAI/resnet18_checkpoint_355.pth', map_location=torch.device("cpu")))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()


In [ ]:
"""
This wrapper computes the segmentation model output and sums the pixel scores for
all pixels predicted as each class, returning a tensor with a single value for
each class. This makes it easier to attribute with respect to a single output
scalar, as opposed to an individual pixel output attribution.
"""
def agg_segmentation_wrapper(inp):
    model_out = model(inp)[0]
    #print(model_out)
    # Creates binary matrix with 1 for original argmax class for each pixel
    # and 0 otherwise. Note that this may change when the input is ablated
    # so we use the original argmax predicted above, out_max.
    out_max = torch.argmax(model_out["masks"], dim=1, keepdim=True)
    selected_inds = torch.zeros_like(model_out["masks"]).scatter_(1, out_max, 1)
    return (model_out["masks"] * selected_inds).sum(dim=(2,3))

    #Original
    # return (model_out["masks"][0] * selected_inds).sum(dim=(2,3))


# Alternate wrapper, simply summing each output channel
def wrapper(inp):
    return model(inp)[0]['masks'].sum(dim=(2,3))

In [ ]:

def decode_segmap(image, nc=21, threshold=0.5):

    label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=cabeça, 2=nadadeiras, 3=corpo, 4=?
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (128, 128, 0)])


    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)

    for l in range(0, nc):
        idx = image > threshold
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]

    rgb = np.stack([r, g, b], axis=2)
    return rgb

### Calculos


In [ ]:
import csv


def CalcularIndices(copia_img, mask_cam, nome, cont, i):



      Image.fromarray(np.uint8(copia_img)).save("/content/drive/MyDrive/Colab Notebooks/XAI/MapadeSaliencia_LayerGradCAM/Aleatorio/peixe"+str(cont)+"_mapa_layer_aleatorio"+str(i)+".png")

      # print("peixe"+str(cont)+"_gradcam_preto"+str(i)+".png")

      img_mask = Image.open("/content/drive/MyDrive/Colab Notebooks/XAI/Máscaras peixes/"+nome+".png")
      mascara_original = np.asarray(img_mask)

      #intersecção sobre a união
      a = np.uint8(mask_cam)
      b = np.uint8(mascara_original)


      mask1_area = np.count_nonzero(a == 1)
      mask2_area = np.count_nonzero(b == 1)
      intersection = np.count_nonzero(np.logical_and(a==1,  b==1))
      iou = intersection/(mask1_area+mask2_area-intersection)

      # print(iou)


      #sorense dice
      dice = np.sum(mask_cam[mascara_original==1])*2.0 / (np.sum(mask_cam) + np.sum(mascara_original))

      # print(dice)


      with open("/content/drive/MyDrive/Colab Notebooks/XAI/MapadeSaliencia_LayerGradCAM/Aleatorio/indices_mapa_layer_aleatorio.csv", "a", newline="") as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=";", quotechar="|", quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        spamwriter.writerow([cont, nome, str(iou), str(dice), i])






In [ ]:
# from prediction_utils import URLDataset, get_maskrcnn_model, get_transform, display_pred, display_black_mask, display_white_mask, get_coloured_mask, check_resolutions
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from time import time
import numpy as np
import cv2
import torch
from torchvision.models import resnet50

def calc_gradcam(image):


      output2 = model([F.to_tensor(image)])[0]


      segmodel = SegmentationModelOutputWrapper(model)
      output = segmodel(image)
      output[0].shape


      #máscaras da predição

      # output = segmodel(image)
      normalized_masks = torch.nn.functional.softmax(output, dim=1).cpu()

      sem_classes = [
          'nadadeiras','corpo', 'cabeca'
      ]
      sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)}

      copia_img1 =  np.float32(image.convert("RGB"))

      mascaras_todas = np.zeros((copia_img1.shape[0], copia_img1.shape[1]))

      for p in output2['masks']:
        t = p.squeeze().detach().cpu().numpy()
        for i in range(len(t)):
          for j in range(len(t[i])):
              if t[i][j]>0.5:
                mascaras_todas[i][j] = 1



      car_mask = torch.from_numpy(mascaras_todas)
      mask = car_mask.clone().detach().numpy()

      #aplicacao grad-cam

      target_layers = [model.backbone[7]]

      targets = [SemanticSegmentationTarget(None, mascaras_todas)]



      with GradCAM(model=model, target_layers=target_layers) as cam:
          grayscale_cam = cam(input_tensor=F.to_tensor(image).unsqueeze(dim=0),targets=targets)[0, :]
          rgb_img = np.float32(image.convert("RGB")) / 255


          grayscale_cam = np.moveaxis(grayscale_cam, 0, 0)

          cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)


          # #Pertubação de pixels
          copia_img1 =  np.float32(image.convert("RGB"))

          # #Matriz zerada da imagem
          mask_cam2 = np.zeros((copia_img1.shape[0], copia_img1.shape[1]))

          for i in range(len(grayscale_cam)):
            for j in range(len(grayscale_cam[i])):
              if grayscale_cam[i][j]>0.01:
                mask_cam2 [i][j] = 1

          return mask_cam2

In [ ]:
def calc_mapa(image):

      model = get_maskrcnn_model(num_classes=4, FROZEN_BLOCKS=1)


      model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/XAI/resnet18_checkpoint_355.pth', map_location=torch.device("cpu")))


      slc = saliency(image, model)


      copia_img2 =  np.float32(image.convert("RGB"))

      #Matriz zerada da imagem
      mask_cam = np.zeros((copia_img2.shape[0], copia_img2.shape[1]))

      for i in range(len(slc)):
        for j in range(len(slc[i])):
          if slc[i][j]>0.045:
              mask_cam [i][j] = 1



      return mask_cam

In [ ]:
import numpy as np
import cv2
import torch
from torchvision.models import resnet50

def calc_cnnfilters(image):

    model = get_maskrcnn_model(num_classes=4, FROZEN_BLOCKS=1)

    model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/XAI/resnet18_checkpoint_355.pth', map_location=torch.device("cpu")))

    model = model.eval()


    preprocessing = transforms.Compose([transforms.ToTensor()])
    preproc_img = preprocessing(image)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=0., std=1.)
    ])
    imagem = transform(image)


    normalized_inp = preproc_img.unsqueeze(0).to('cpu')
    normalized_inp.requires_grad = True
    out = model(normalized_inp)


    result =  CnnFilters(imagem, model)


    #preparo perturbação

    copia_result = np.zeros((result.shape[0], result.shape[1], 3))

    for i in range(len(result)):
      for j in range(len(result[i])):
        if result[i][j]<0:
          copia_result [i][j][0] = 255
          copia_result [i][j][1] = 0
          copia_result [i][j][2] = 0

    resultado = cv2.resize(copia_result, (image.size[0], image.size[1]), interpolation = cv2.INTER_LINEAR)

    copia_img =  np.float32(image.convert("RGB"))

    #Matriz zerada da imagem
    mask_cam3 = np.zeros((copia_img.shape[0], copia_img.shape[1]))

    for i in range(len(resultado)):
      for j in range(len(resultado[i])):
        if resultado[i][j][0]>128:
          mask_cam3 [i][j] = 1

    return mask_cam3


In [ ]:
def calc_layergradcam(img):


      preprocessing = transforms.Compose([transforms.ToTensor()])
      preproc_img = preprocessing(img)

      normalized_inp = preproc_img.unsqueeze(0).to(device)
      normalized_inp.requires_grad = True

      out = model(normalized_inp)

      #Feito pela Ju - tensor estava voltando zerado
      out_mask = out[0]['masks']
      out_labels = out[0]['labels']


      out_max = torch.argmax(out_mask, dim=1, keepdim=True)

      for mask in out_mask:
        rgb = decode_segmap(mask.detach().cpu().squeeze().numpy(), 3, 0.90)


      lgc = LayerGradCam(agg_segmentation_wrapper, model.backbone[7])
      gc_attr = lgc.attribute(normalized_inp, target=0)

      # la = LayerActivation(wrapper,  model.backbone[7])
      # activation = la.attribute(normalized_inp)
      result = gc_attr[0].cpu().permute(1,2,0).detach().numpy()
      result2 = gc_attr[0].cpu().permute(1,2,0).detach().numpy()
      copia_result2 = np.zeros((result2.shape[0], result2.shape[1], 3))

      for k in range(len(result2)):
        for l in range(len(result2[k])):
          if result2[k][l]>-0.5:
            copia_result2 [k][l][0] = 255
            copia_result2 [k][l][1] = 0
            copia_result2 [k][l][2] = 0

      resultado2 = cv2.resize(copia_result2, (img.size[0],img.size[1]), interpolation = cv2.INTER_LINEAR)


      #Perturbação de pixels
      copia_img2 =  np.float32(img.convert("RGB"))

      #Matriz zerada da imagem
      mask_cam2 = np.zeros((copia_img2.shape[0], copia_img2.shape[1]))

      # #RGB - Preto
      for i in range(len(resultado2)):
       for j in range(len(resultado2[i])):
        if resultado2[i][j][0]>128:
             mask_cam2 [i][j] = 1
             # print(grayscale_cam[i][j])

      return mask_cam2

In [ ]:
cont = 0

for imagens in linhas:
  cont += 1
  nome = imagens[0:25]
  print(nome)
  image = Image.open("/content/drive/MyDrive/Colab Notebooks/XAI/Imagens peixes/"+nome+".png").convert("RGB")
  # image = Image.open("/content/drive/MyDrive/Colab Notebooks/XAI/Imagens peixes/.png").convert("RGB")

  for a in [1,2,3,4,5]:
    try:
      print(a)
      if a>1:
        num = a-1
        image = Image.open("/content/drive/MyDrive/Colab Notebooks/XAI/MapadeSaliencia_LayerGradCAM/Aleatorio/peixe"+str(cont)+"_mapa_layer_aleatorio"+str(num)+".png").convert("RGB")

      # mask1 = calc_gradcam(image)
      # mask1 = calc_cnnfilters(image)
      mask1 = calc_layergradcam(image)
      mask2 = calc_mapa(image)


      mascara_combinada = cv2.addWeighted(mask1, 1, mask2, 1, 0)
      # m = Image.fromarray(np.uint8(mascara_combinada)*255).convert('RGB')
      # m.show()
      # #RGB - Preto

      #Pertubação de pixels
      copia_img =  np.float32(image.convert("RGB"))


      # Preto
      # for i in range(len(mascara_combinada)):
      #   for j in range(len(mascara_combinada[i])):
      #     if mascara_combinada[i][j]!=0:
      #         copia_img [i][j][0] = 0
      #         copia_img [i][j][1] = 0
      #         copia_img [i][j][2] = 0

      # im = Image.fromarray(np.uint8(copia_img))
      # im.show()

      #White Noise

      # for i in range(len(mascara_combinada)):
      #   for j in range(len(mascara_combinada[i])):
      #     if mascara_combinada[i][j]!=0:
      #       rdn = random.random()
      #       if rdn < 0.5:
      #         copia_img [i][j][0] = 0
      #         copia_img [i][j][1] = 0
      #         copia_img [i][j][2] = 0
      #       elif rdn > 0.5:
      #         copia_img [i][j][0] = 255
      #         copia_img [i][j][1] = 255
      #         copia_img [i][j][2] = 255
              # print(grayscale_cam[i][j])

      #Pixels Aleatórios


      for i in range(len(mascara_combinada)):
        for j in range(len(mascara_combinada[i])):
          if mascara_combinada[i][j]!=0:
            copia_img [i][j][0] = random.randint(0, 255)
            copia_img [i][j][1] = random.randint(0, 255)
            copia_img [i][j][2] = random.randint(0, 255)





      CalcularIndices(copia_img, mascara_combinada, nome, cont, a)

    except:
        break



ckz4dow5r8eay0z9i9oef91zg
1
2
ckz4e0jvq8h73109bbpqs0fps
1
2
ckz4e2n4q8f530z8n9s3ubp7y
1
2
ckz4e3onq8gmi0z9i5c70ay45
1
2
ckz4e4vvo8i6i109bgzla6hms
1
2
ckz4e8ge48hhl0z9ifd9773xk
1
2
ckz4e9ebq8gg20z8ngztba1ez
1
2
ckz4e650k8ig1109b97k528nn
1
2
ckz4e72918ion109b5wbr5raw
1
2
ckz4ea3cz8jbf109bhn9y68q1
1
2
ckz4ebk6g8i8u0z9i60n60f6l
1
2
ckz4edumg8ing0z9ifni9g5t8
1
2
ckz4efrf38kem109b2057fb9a
1
2
ckz4egfr48kh7109bah7xf5az
1
2
ckz4fduq78ntw0z8n630rag87
1
2
ckz4ff6t28o3e0z8neshr668d
1
2
ckz4ffyse8o9s0z8n3zgmd73e
1
2
ckz4fhxu88ool0z8n6rwr1cpm
1
2
ckz4fiupe8rhx109bh3peeyhc
1
2
ckz4fzcn28sui0z8nakiacny4
1
2
ckz4g2msm8tqx0z8n6d9pb3iv
1
2
ckz4g4h098uag0z8nht9q26i8
1
2
ckz4g5tsc8um20z8n9i7a5xw6
1
2
ckz4g6h9v8x8e109b16pefhfo
1
2
ckz4g7nl08v2x0z8nbuek2vjk
1
2
ckz4g09s18vbm109bhm9pcd53
1
2
ckz4g9uib8x7y0z9i2x6dgzf8
1
2
ckz4g17ra8upt0z9ibj0bbdno
1
2
ckz4g51wk8wtj109bhzrydupk
1
2
ckz4g70yq8we70z9idki606ak
1
2
ckz4g39038vcp0z9i67o96efj
1
2
ckz4g88978v9h0z8n0bmhf6m2
1
2
ckz4gb3qw8yne109bd7k39lcl
1
2
ckz4gbrb08